#  Intracranial Hemorrhage Detection

## Model 5

#### Kristina Joos

---   

|                 	|                                                        	|
|:----------------:	|:-------------------------------------------------------:	|
| Model           	| VGG 16 (242x242)                                          |
| Preprocessing   	| Augementation Flip, Single Windowing                   	|
| Class Balancing 	| Oversampling                                           	|
| Loss Function   	| Binary_crossentropy                                      	|
| Regularization  	| Early Stopping, Drop Out Layers 0.3 	                    |
| Epochs Run      	| 50                                                     	|
| Time Run (min)   	| 60                                                       	|
|                 	|                                                        	|
| Test Sores      	| Accuracy: Loss:                                        	|
| Validation      	| Accuracy: Loss:                                        	|
| Leader Board    	| Score: Rank:                                           	|




In [100]:
import numpy as np
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import collections
from tqdm import tqdm_notebook as tqdm
from datetime import datetime

from math import ceil, floor, log
import cv2

import tensorflow as tf
#import tensorflow.keras
from tensorflow.keras import backend as K

import sys

# from keras_applications.resnet import ResNet50
from keras_applications.inception_v3 import InceptionV3




from skimage.transform import resize
from imgaug import augmenters as iaa
 
from sklearn.model_selection import train_test_split, StratifiedKFold, ShuffleSplit



from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet_50
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg_16

from tensorflow.keras.applications import ResNet50, VGG16, VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet_50
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg_16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Activation, concatenate, Dropout, MaxPooling2D, Conv2D, Flatten
from tensorflow.keras.initializers import glorot_normal, he_normal
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model, load_model, Sequential, load_model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam

from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit


#from tensorflow.nn import sigmoid_cross_entropy_with_logits



In [101]:
tf.test.is_gpu_available()

True

In [102]:
tf.test.is_built_with_cuda()

True

## Image Path

In [103]:
path_train_data = '../data/input/rsna-intracranial-hemorrhage-detection/stage_2_train/'

In [104]:
path_test_data = '../data/input/rsna-intracranial-hemorrhage-detection/stage_2_test/'

In [105]:
def get_number_of_files(path, data_name):
    no =  len(os.listdir(path))
    print (f'The {data_name} contains {no} files.')

In [106]:
def get_size_directory(path, data_name):
    size = round(sum([os.path.getsize(f'{path}'+ f'{file}') for file in os.listdir(path)])*(10**-9), 2)
    print (f'The size of the {data_name} is {size} GB.')

In [107]:
print('***TRAIN DATA***')
get_size_directory(path_train_data, 'Training Data')
get_number_of_files(path_train_data, 'Training Data')

print('*******')

print('***TEST DATA***')
get_size_directory(path_test_data, 'Testing Data')
get_number_of_files(path_test_data, 'Testing Data')

print('*******')

print(f'We have {round(len(os.listdir(path_train_data))/len(os.listdir(path_test_data)),1)} times more train than test data.' )

***TRAIN DATA***
The size of the Training Data is 395.19 GB.
The Training Data contains 752803 files.
*******
***TEST DATA***
The size of the Testing Data is 63.64 GB.
The Testing Data contains 121232 files.
*******
We have 6.2 times more train than test data.


## Meta Data

In [108]:

train_meta_df = pd.read_csv('../data/input/rsna-intracranial-hemorrhage-detection/stage_2_train.csv')
# Test submission as test 
test_meta_df = pd.read_csv('../data/input/rsna-intracranial-hemorrhage-detection/stage_2_sample_submission.csv')

In [109]:
train_meta_df_shape = train_meta_df.shape
test_meta_df_shape = test_meta_df.shape

In [110]:
print(train_meta_df_shape)
print(test_meta_df_shape)

(4516842, 2)
(727392, 2)


## Parameters

In [111]:

# Seed
SEED = 11
np.random.seed(SEED)


# Constants
TEST_SIZE = 0.2
HEIGHT = 224 #VGG 16 has 256x256
WIDTH = 224
CHANNELS = 3
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2
TEST_BATCH_SIZE = 4
SHAPE = (HEIGHT, WIDTH, CHANNELS)

# Folders
#DATA_DIR = '/kaggle/input/rsna-intracranial-hemorrhage-detection/'
PATH_TEST_DATA = path_test_data
PATH_TRAIN_DATA = path_train_data
PATH_PT_MODELS = '../models/predtrained models/'

In [112]:
def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(dcm, window_center, window_width):    
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    
    # Resize
    img = cv2.resize(img, SHAPE[:2], interpolation = cv2.INTER_LINEAR)
   
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img

def bsb_window(dcm):
    brain_img = window_image(dcm, 40, 80)
    subdural_img = window_image(dcm, 80, 200)
    soft_img = window_image(dcm, 40, 380)
    
    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380
    bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)
    return bsb_img

def _read(path, SHAPE):
    dcm = pydicom.dcmread(path)
    try:
        img = bsb_window(dcm)
    except:
        img = np.zeros(SHAPE)
    return img

In [113]:
# Image Augmentation
sometimes = lambda aug: iaa.Sometimes(0.25, aug)
augmentation = iaa.Sequential([ iaa.Fliplr(0.25),
                                iaa.Flipud(0.10),
                                sometimes(iaa.Crop(px=(0, 25), keep_size = True, sample_independently = False))   
                            ], random_order = True)       
        
# Generators
class TrainDataGenerator(Sequence):
    def __init__(self, dataset, labels, batch_size = 16, img_size = SHAPE, img_dir = PATH_TRAIN_DATA, augment = False, *args, **kwargs):
        self.dataset = dataset
        self.ids = dataset.index
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.augment = augment
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.ids) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        X, Y = self.__data_generation(indices)
        return X, Y

    def augmentor(self, image):
        augment_img = augmentation        
        image_aug = augment_img.augment_image(image)
        return image_aug

    def on_epoch_end(self):
        self.indices = np.arange(len(self.ids))
        np.random.shuffle(self.indices)

    def __data_generation(self, indices):
        X = np.empty((self.batch_size, *self.img_size))
        Y = np.empty((self.batch_size, 6), dtype=np.float32)
        
        for i, index in enumerate(indices):
            ID = self.ids[index]
            image = _read(self.img_dir+ID+".dcm", self.img_size)########
            if self.augment:
                X[i,] = self.augmentor(image)
            else:
                X[i,] = image
            Y[i,] = self.labels.iloc[index].values        
        return X, Y
    
class TestDataGenerator(Sequence):
    def __init__(self, dataset, labels, batch_size = 16, img_size = SHAPE, img_dir = PATH_TEST_DATA, *args, **kwargs):
        self.dataset = dataset
        self.ids = dataset.index
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = test_dir
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.ids) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        X = self.__data_generation(indices)
        return X

    def on_epoch_end(self):
        self.indices = np.arange(len(self.ids))
    
    def __data_generation(self, indices):
        X = np.empty((self.batch_size, *self.img_size))
        
        for i, index in enumerate(indices):
            ID = self.ids[index]
            image = _read(self.img_dir+ID+".dcm", self.img_size)########
            X[i,] = image              
        return X

## Preparing Meta Data Frames


In [114]:
def make_usable_df(df):
    label = df.Label.values
    new_df = df.ID.str.rsplit("_", n=1, expand=True)
    new_df.loc[:, "label"] = label
    new_df = new_df.rename({0: "id", 1: "subtype"}, axis=1)
    piv_df = pd.pivot_table(new_df, index="id", columns="subtype", values="label")
    
    return piv_df


Check for Dupicates:

In [115]:
train_meta_df.duplicated().sum()

24

In [116]:
train_meta_df.drop_duplicates(inplace = True)

In [117]:
train_meta_df_shape[0] - train_meta_df.shape[0] == 24

True

Make usable Data Frame:

In [118]:
train_df = make_usable_df(train_meta_df)
train_df.head()

subtype,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
id,,,,,,
ID_000012eaf,0,0,0,0,0,0
ID_000039fa0,0,0,0,0,0,0
ID_00005679d,0,0,0,0,0,0
ID_00008ce3c,0,0,0,0,0,0
ID_0000950d7,0,0,0,0,0,0



Check for Duplicates:

In [119]:
test_meta_df.duplicated().sum()

0

Make usable Data Frame:

In [120]:
test_df = make_usable_df(test_meta_df)
test_df.head()

subtype,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
id,,,,,,
ID_000000e27,0.5,0.5,0.5,0.5,0.5,0.5
ID_000009146,0.5,0.5,0.5,0.5,0.5,0.5
ID_00007b8cb,0.5,0.5,0.5,0.5,0.5,0.5
ID_000134952,0.5,0.5,0.5,0.5,0.5,0.5
ID_000176f2a,0.5,0.5,0.5,0.5,0.5,0.5


# Train/ Val Split

## Multi Label Stratified Shuffel Splitter 
Cross Validaor with stratification on multiple labels:
https://github.com/trent-b/iterative-stratification

In [121]:
msss = MultilabelStratifiedShuffleSplit(n_splits = 10, test_size = TEST_SIZE, random_state = SEED)
X = train_df.index
Y = train_df[['any', 'epidural', 'intraparenchymal', 'intraventricular',
       'subarachnoid', 'subdural']].values

# Get train and test index
msss_splits = next(msss.split(X, Y))
train_idx = msss_splits[0]
val_idx = msss_splits[1]

In [122]:
train_idx.shape

(602242,)

In [123]:
val_idx.shape

(150561,)

In [124]:
train_data = train_df.iloc[train_idx]

In [125]:
train_data.shape

(602242, 6)

In [126]:
val_data = train_df.iloc[val_idx]

In [127]:
val_data.shape

(150561, 6)

## Custom Loan Functions

In [128]:
from tensorflow.python.ops import array_ops

In [129]:
def focal_loss(prediction_tensor, target_tensor, weights=None, alpha=0.25, gamma=2):
    r"""Compute focal loss for predictions.
        Multi-labels Focal loss formula:
            FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log(1-p)
                 ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
    Args:
     prediction_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing the predicted logits for each class
     target_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing one-hot encoded classification targets
     weights: A float tensor of shape [batch_size, num_anchors]
     alpha: A scalar tensor for focal loss alpha hyper-parameter
     gamma: A scalar tensor for focal loss gamma hyper-parameter
    Returns:
        loss: A (scalar) tensor representing the value of the loss function
    """
    sigmoid_p = tf.nn.sigmoid(prediction_tensor)
    zeros = array_ops.zeros_like(sigmoid_p, dtype=sigmoid_p.dtype)
    
    # For poitive prediction, only need consider front part loss, back part is 0;
    # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
    pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)
    
    # For negative prediction, only need consider back part loss, front part is 0;
    # target_tensor > zeros <=> z=1, so negative coefficient = 0.
    neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
    per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * K.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                          - (1 - alpha) * (neg_p_sub ** gamma) * K.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
    return tf.reduce_sum(per_entry_cross_ent)

## Make Model


In [130]:
def vgg_16():
    weights_path = '../models/predtrained models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
    net = VGG16(include_top=False, weights=weights_path)
    #for layer in net.layers:
        #layer.trainable = False
    return net

In [131]:
my_vgg_16 = vgg_16()

In [132]:
class MakeModel:
    
    def __init__(self,
                 pt_model,
                 #loss,
                 metrics_list,
                 data_generator_train,
                 data_generator_val,
                 epochs,
                 num_classes=6,
                 checkpoint_path='../models/mymodels/'):
        
        self.pt_model = pt_model
        #self.loss = loss
        self.metrics_list = metrics_list
        self.data_generator_train = data_generator_train
        self.data_generator_val = data_generator_val
        self.epochs = epochs
        self.num_classes = num_classes
        self.checkpoint_path = checkpoint_path 
        self.checkpoint = ModelCheckpoint(filepath=self.checkpoint_path,
                                          mode="min",
                                          verbose=1,
                                          save_best_only=True,
                                          save_weights_only=True)
        self.reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                           factor=0.5,
                                           patience=2,
                                           min_lr=1e-8,
                                            mode="min")
        self.earlystopping = EarlyStopping(monitor="val_loss",
                                        patience=5,
                                        mode="min",
                                        restore_best_weights=True)
        
    def build_model(self):
        base_model = self.pt_model
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dropout(0.5)(x)
        any_logits = Dense(1, kernel_initializer=he_normal(seed=11))(x)
        any_pred = Activation("sigmoid", name="any_predictions")(any_logits)
        x = concatenate([any_pred, x])
        sub_pred = Dense(self.num_classes,
                         name="subtype_pred",
                         kernel_initializer=he_normal(seed=11),
                         activation="sigmoid")(x) 
        self.model = Model(inputs=base_model.input, outputs=[any_pred, sub_pred])
        
        
    
    def compile_model(self):
        self.model.compile(optimizer=Adam(0.0001),
                           loss=['binary_crossentropy', focal_loss],
                           loss_weights = [1., 0.],
                           metrics=self.metrics_list)
    
    def fit_model(self):
        return self.model.fit_generator(generator = self.data_generator_train,
                    validation_data=self.data_generator_val,
                    epochs=self.epochs,
                    callbacks=[self.checkpoint,self.earlystopping],
                    use_multiprocessing=True,
                    workers=-1)
    
    def load_weights(self, path):
        self.model.load_weights(path)
    
    def predict(self, data_generator_test):
        predictions = self.model.predict_generator(data_generator_test, use_multiprocessing = True, workers=-1)
        return predictions

In [133]:
train_preprocessor = Preprocessor(path=PATH_TRAIN_DATA,
                                  #hu_min_value=0,
                                  #hu_max_value=100,
                                  augment=False)

In [134]:
val_preprocessor = Preprocessor(path=PATH_TRAIN_DATA,
                                  #hu_min_value=0,
                                  #hu_max_value=100,
                                  augment=False)

In [135]:
test_preprocessor = Preprocessor(path=PATH_TEST_DATA,
                                #hu_min_value=0,
                                #hu_max_value=100,
                                augment=False)

In [136]:
data_generator_train = TrainDataGenerator(train_df.iloc[train_idx], 
                                                train_df.iloc[train_idx], 
                                                TRAIN_BATCH_SIZE, 
                                                SHAPE,
                                                augment = True)

data_generator_val = TrainDataGenerator(train_df.iloc[val_idx], 
                                            train_df.iloc[val_idx], 
                                            VALID_BATCH_SIZE, 
                                            SHAPE,
                                            augment = False)


In [137]:
def predictions(test_df, model):    
    test_preds = model.predict_generator(TestDataGenerator(test_df, None, 5, SHAPE, TEST_IMAGES_DIR), verbose = 1)
    return test_preds[:test_df.iloc[range(test_df.shape[0])].shape[0]]


In [138]:
train_data.loc[data_generator_train.data_ids].sum() / train_data.loc[data_generator_train.data_ids].shape[0]

AttributeError: 'TrainDataGenerator' object has no attribute 'data_ids'

In [139]:
model5 = MakeModel(pt_model=my_vgg_16,
                      #loss="binary_crossentropy", #multilabel_focal_loss(class_weights=my_class_weights, alpha=0.5, gamma=0),
                      metrics_list={'any_predictions':['binary_crossentropy', 'accuracy'],
                                            'subtype_pred': [focal_loss, 'accuracy']},
                      data_generator_train = data_generator_train,
                      data_generator_val = data_generator_val,
                      epochs=200,
                      num_classes=5)
model5.build_model()
model5.compile_model()
model5_history = model5.fit_model()


Epoch 1/200


ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 1 arrays: [array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)]...

In [ ]:
model2.model.summary()

In [ ]:
preds = predictions(test_df, model5)

In [ ]:
train_loss = model3_history.history['any_predictions_loss']
val_loss = model3_history.history['val_any_predictions_loss']

plt.figure(figsize = (12,8))

plt.plot(train_loss, label = 'training_loss', color = 'b')
plt.plot(val_loss, label = 'val_loss', color = 'orange')

plt.title('Any Label Training and Validation Loss by Epochs')
plt.xlabel('Epochs')
plt.ylabel('weighted_log_loss')
plt.legend();

In [ ]:
train_loss = model3_history.history['subtype_pred_loss']
val_loss = model3_history.history['val_subtype_pred_loss']

plt.figure(figsize = (12,8))

plt.plot(train_loss, label = 'training_loss', color = 'b')
plt.plot(val_loss, label = 'val_loss', color = 'orange')

plt.title('Subtype Label Training and Validation Loss by Epochs')
plt.xlabel('Epochs')
plt.ylabel('weighted_log_loss')
plt.legend();

In [ ]:
train_loss = model3_history.history['subtype_pred_focal_loss']
val_loss = model3_history.history['val_subtype_pred_focal_loss']

plt.figure(figsize = (12,8))

plt.plot(train_loss, label = 'training_loss', color = 'b')
plt.plot(val_loss, label = 'val_loss', color = 'orange')

plt.title('Any Label Training and Validation Focal Loss by Epochs')
plt.xlabel('Epochs')
plt.ylabel('weighted_log_loss')
plt.legend();

In [ ]:
train_acc = model3_history.history['subtype_pred_accuracy']
val_acc = model3_history.history['val_subtype_pred_accuracy']

plt.figure(figsize = (12,8))

plt.plot(train_loss, label = 'training_acc', color = 'b')
plt.plot(val_loss, label = 'val_acc', color = 'orange')

plt.title('Training and Validation Subtype Accuracy by Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend();

In [ ]:
train_proba_any, train_proba_subtype = model3.predict(data_generator_test)

In [ ]:
train_proba_any.shape

In [ ]:
train_proba_subtype.shape

In [ ]:
submission = np.concatenate((train_proba_any,train_proba_subtype),axis=1)

In [ ]:
submission.shape

In [127]:
def make_df(meta_data_df, predictions):
    df = pd.DataFrame(predictions, columns=meta_data_df.columns, index=meta_data_df.index)
    df = df.stack().reset_index()
    df.loc[:, "ID"] = df.id.str.cat(df.subtype, sep="_")
    df = df.drop(["id", "subtype"], axis=1)
    df = df.rename({0: "Label"}, axis=1)
    df = df[['ID','Label']]
   
    return df

In [128]:
submission_5 = make_df(test_df, submission)

In [131]:
submission_5.to_csv('../data/output/submissions/submission_model_5.csv', index=False)